# Imports

In [1]:
from collections import defaultdict

# The Classes

In [2]:
class BinaryTree:
    ''' A class for creating and handling binary trees '''
    
    def __init__(self, nodes = []):
        self.nodes = nodes

    def root(self):
        return self.nodes[0]
    
    def iparent(self, i):
        ''' Returns the parent of the node i '''
        return (i - 1) // 2
    
    def ileft(self, i):
        ''' Returns the left child of i '''
        return 2*i + 1

    def iright(self, i):
        ''' Returns the right child of i '''
        return 2*i + 2

    def left(self, i):
        ''' Returns the positions of the left child of i '''
        return self.node_at_index(self.ileft(i))
    
    def right(self, i):
        ''' Returns the positions of the right child of i '''
        return self.node_at_index(self.iright(i))

    def parent(self, i):
        ''' Returns the positions of the parent of i '''
        return self.node_at_index(self.iparent(i))

    def node_at_index(self, i):
        ''' Find out whick element is in the position i '''
        return self.nodes[i]

    def size(self):
        ''' Returns the size of the tree'''
        return len(self.nodes)
    

In [3]:
class MinHeap(BinaryTree):
    ''' A Binary Tree where the root is the smallest number and every child follows by ascending value '''

    def __init__(self, nodes):
        BinaryTree.__init__(self, nodes)
        self.min_heapify()
        # self.nodes all the elements in my heap

    # Heapify at a node assuming all subtrees are heapified
    def min_heapify_subtree(self, i):
        size = self.size() # size of the tree
        ileft = self.ileft(i) # Calculate what woulf be the left child of i
        iright = self.iright(i) # Calculate what would be the Right child of i 
        imin = i 
        if( ileft < size and self.nodes[ileft] < self.nodes[imin]):
            '''
            if the calculated left child IS in the tree and
            its position is ahead of the min(the parent), then
            say that the minimum is the left child
            '''
            imin = ileft
            
        if( iright < size and self.nodes[iright] < self.nodes[imin]): 
            '''
            if the calculated right child IS in the tree and
            its position is ahead of the min(the parent), then
            say that the minimum is the right child
            '''
            imin = iright
            
        if( imin != i):  
            '''
            If it turns out that the parent is in fact larger
            than the chidren (we've swapped the min with some of
            its children), then swap also their place in the tree
            Once you do that check if the new "parent" (the child
            became a parent) is in the right position
            '''
            
            self.nodes[i], self.nodes[imin] = self.nodes[imin], self.nodes[i]
            self.min_heapify_subtree(imin)                          


    def min_heapify(self):
        '''
        Iterate through every node of the heap from the last to the first 
        and make our heap with the node with the minimu value is the root 
        and all the other nodes follow in ascending order
        '''
        for i in range(len(self.nodes), -1, -1): 
            self.min_heapify_subtree(i)

    def min(self):
        return self.nodes[0] # Return the root, aka the minimum

    def pop(self):
        '''
        Find the minimum element in the heap (tree) aka the root. If the tree is not just this element remove it
        and replace it with the last object in the heap. Run min_heapify to reorder the heap(maintain the heap property). 
        '''
        min = self.nodes[0]
        if self.size() > 1:
            self.nodes[0] = self.nodes[-1]
            self.nodes.pop()
            self.min_heapify_subtree(0)
        elif self.size() == 1: 
            self.nodes.pop()
        else:
            return None
        return min

    def decrease_key(self, element, val):
        '''
        Find the position of the node i and change its value.
        Now check if with the changed value the position of i 
        has the heap property
        '''
        for i in range(len(self.nodes)):
            if element == self.nodes[i][1]:
                self.nodes[i][0] = val
                index = i
                break
        #self.nodes[i] = val
        iparent = self.iparent(i)
        while( i != 0 and self.nodes[iparent] > self.nodes[i]):
            self.nodes[iparent], self.nodes[i] = self.nodes[i], self.nodes[iparent]
            i = iparent
            if i > 0:
                iparent = self.iparent(i)

In [4]:
class Graph(object):
    '''
    Graph Data Structure, undirected by default
    '''
    
    def __init__(self):
        self.adjacency = defaultdict(set)
        self.dis = {}
        self.tim = {}
        self.net = {}
        self.coordinates = {}
        self.adj_list_d = defaultdict(list)
        self.adj_list_t = defaultdict(list)

    def add(self, node1, node2):
        ''' Add connection between node1, node2'''
        self.adjacency[int(node1)].add(int(node2))
        self.adjacency[int(node2)].add(int(node1))
        self.net[(int(node1), int(node2))] = 1
        
    def distance(self, node1, node2, d):
        ''' Create the distance measure between node1 and node2 '''
        self.dis[(int(node1), int(node2))] = int(d)
        self.adj_list_d[int(node1)].append((int(d), int(node2)))
        self.adj_list_d[int(node2)].append((int(d), int(node1)))
        
    def time(self, node1, node2, t):
        ''' Create the time distance measure between node 1 and node 2'''
        
        self.tim[(int(node1), int(node2))] = int(t)
        self.adj_list_t[int(node1)].append(( int(t), int(node2)))
        self.adj_list_t[int(node2)].append((int(t), int(node1)))
        
    def coordinate(self, node, coordinate1, coordinate2):
        '''Save the coordinates of every node'''
        self.coordinates[int(node)] = [int(coordinate1), int(coordinate2)]
    
    def nodes_(self):
        ''' All the nodes of the graph '''
        return list(set(self.adjacency.keys()))
    
    def edges(self):
        ''' Return all the edges of the graph '''
        return list(self.dis.keys())
    
    def print_adj_list_d(self):
        for keys, values in self.adj_list_d.items():
            print(keys, values)
            
    def print_adj_list_t(self):
        for keys, values in self.adj_list_d.items():
            print( keys, values)
            
    def dijksta(self, v, measurement):
        ''' Returns the shortest distance of v and all the other nodes'''
        distance_from_v = {ele : float("inf") for ele in self.nodes_()}
        distance_from_v[v] = 0
        heap = MinHeap([[values, keys] for keys, values in distance_from_v.items()])
        seen = []
        edges_ = defaultdict()
        while heap.size() > 0:
            c_n = heap.pop()[1]
            seen.append(c_n)
            for n in self.adjacency[c_n].difference(set(seen)):
                if distance_from_v[n] > distance_from_v[c_n] + measurement[(c_n, n)]:
                    if heap.size()>0:
                        if c_n in edges_.keys():
                            edges_[n] = edges_[c_n] +  [(c_n, n)]
                        else: 
                            edges_[n] = [(c_n, n)]
                    distance_from_v[n] = distance_from_v[c_n] + measurement[(c_n, n)]
                    heap.decrease_key(n,distance_from_v[n])
        return distance_from_v, edges_
    
    def functionality1(self):
        ''' 
        Input : - set of nodes v = {v_1, ...., v_n}
                - a distance threshold d
        Output : set of nodes at distance <= d from v
        '''
        v = int(input("Give me the node "))
        measure = input("The function to calculate the distance ")
        d = int(input(" and the threshold distance "))
        if measure.lower() == "distance":
            neigh = self.dijksta(v, self.dis)[0]
        elif measure.lower() == "time" or measure.lower() == "time distance" :
            neigh = self.dijksta(v, self.tim)[0]
        elif measure.lower() == "network" or measure.lower() == "network distance":
            neigh = self.dijksta(v, self.net)[0]
        p = []
        for keys, values in neigh.items():
            if keys == v:
                continue
            if values <=d:
                p.append(keys)
        return p
    
    def functionality2(self):
        '''
        Input : a set of nodes v = {v_1, ..., v_n}
        Output : the set of roads (edges) that enable the user to visit all the places
        '''
        v_set = set(map(int,input("Give me the set of nodes ").split()))
        measure = input("The function to calculate the distance ")
        if measure.lower() == "distance":
            measure = self.dis
        elif measure.lower() == "time" or measure.lower() == "time distance" :
            measure = self.tim
        elif measure.lower() == "network" or measure.lower() == "network distance":
            measure = self.net
        cost_ = defaultdict(list)
        for i in v_set:
            yo = {ele : float("inf") for ele in v_set}
            yo[i] = 0
            heap = MinHeap([[values, keys] for keys, values in yo.items()])
            seen_ = []
            edges__ = []
            cost = 0
            while heap.size() > 0:
                c_n = heap.pop()[1]
                seen_.append(c_n)
                for n in v_set.difference(set(seen_)):
                    if yo[n] > yo[c_n] + G.dijksta(c_n, measure)[0][n]:
                        yo[n] = yo[c_n] + G.dijksta(c_n, measure)[0][n]
                        heap.decrease_key(n,yo[n])
                        edges__ = edges__ +  G.dijksta(c_n, measure)[1][n]
                if heap.size() > 0:
                    cost = cost + G.dijksta(c_n, measure)[0][heap.min()[1]]
            cost_[cost] = edges__
        return min(cost_.items(), key=lambda x: x[0])[1]

# Read the Data
#### Previewing the data, we notice that the first 7 rows are an introduction and an a prologue for the data that follows, so we skip these

#### Let's create a Graph out of our Data

In [5]:
G = Graph()

In [6]:
with open(r"C:\Users\HP\Documents\ADM\HW 5\USA-road-d.CAL.gr", encoding='utf-8') as file:
    n = 0
    for line in file:
        if n > 6 and n<100:
            ww = line.split()
            G.add(ww[1], ww[2])
            G.distance(ww[1], ww[2], ww[3])
        n += 1

In [7]:
with open(r"C:\Users\HP\Documents\ADM\HW 5\USA-road-t.CAL.gr", encoding='utf-8') as file:
    n = 0
    for line in file:
        if n > 6 and n<100:
            ww = line.split()
            G.add(ww[1], ww[2])
            G.time(ww[1], ww[2], ww[3])
        n += 1

In [8]:
with open(r"C:\Users\HP\Documents\ADM\HW 5\USA-road-co.CAL.gr", encoding='utf-8') as file:
    n = 0
    for line in file:
        if n > 6 :
            ww = line.split()
            G.coordinate(ww[1], ww[2], ww[3])
        n += 1

In [10]:
G.functionality1()

Give me the node 1
The function to calculate the distance distance
 and the threshold distance 300000


[1048577]

In [14]:
G.functionality2()

Give me the set of nodes 1 1048577
The function to calculate the distance distance


[(1048577, 1)]